In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
data=pd.read_csv("Fake.csv")
data.head()

In [ ]:
true = pd.read_csv("Fake.csv")
false = pd.read_csv("Fake.csv")


In [ ]:
true.head()

In [ ]:
false.head()

In [ ]:
true['category'] = 1
false['category'] = 0

In [ ]:
true.head()

In [ ]:
df = pd.concat([true,false]) 

In [ ]:
df.head()

In [ ]:
df.isna().sum()
df.title.count()
df.subject.value_counts()
df.head()

In [ ]:
df['text'] = df['text'] + " " + df['title'] + " " + df['subject']
del df['title']
del df['subject']
del df['date']

In [ ]:
df.head()

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
stop

In [ ]:
stemmer = PorterStemmer()
def stem_text(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            word = stemmer.stem(i.strip())
            final_text.append(word)
    return " ".join(final_text) 

In [ ]:
df.text = df.text.apply(stem_text)

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 3000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df.text))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.category)


In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

In [ ]:
y_train

In [ ]:
print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

In [ ]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])


In [ ]:
batch_size = 32
num_samples = cv_train_reviews.shape[0]
num_batches = num_samples // batch_size

for epoch in range(5):
    print(f"Epoch {epoch+1}/{5}")
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        x_batch = cv_train_reviews[start:end]
        y_batch = y_train[start:end]

        model.train_on_batch(x_batch, y_batch)

In [ ]:
[model.fit(cv_train_reviews, y_train, epochs=5)]

In [ ]:
from joblib import dump, load
dump(model, 'CNN.model') 

In [ ]:
pred = (model.predict(cv_test_reviews) > 0.9999)

In [ ]:
accuracy_score(pred,y_test)


In [ ]:
accuracy_score(pred,y_test)

In [ ]:
accuracy_score(pred,y_test)

In [ ]:
cm_cv = confusion_matrix(pred,y_test)
cm_cv

In [ ]:
cm_cv = confusion_matrix(pred,y_test)
cm_cv

In [ ]:
cm_cv1 = confusion_matrix(pred,y_test)
cm_cv1

In [ ]:
cm_cv2 = confusion_matrix(y_test,pred)
cm_cv2

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm_cv,cmap= "Blues",annot = True, fmt='')

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm_cv,cmap= "Blues",annot = True, fmt='')

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm_cv1,cmap= "Blues",annot = True, fmt='')
